In [ ]:
import time
from io import BytesIO
from datetime import datetime

import pandas as pd
import win32clipboard
from PIL import Image
from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("disable-infobars")  # Deshabilitar infobars
options.add_argument("--disable-notifications")  # Deshabilitar notifications

options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches", ["enable-automation"])

In [ ]:
cService = webdriver.ChromeService(executable_path='./drivers/chromedriver.exe')
browser = webdriver.Chrome(service=cService)

In [ ]:
browser.get("https://web.whatsapp.com/")

In [ ]:
def paste_content(driver, el, content):
    driver.execute_script(
        f'''
        const text = `{content}`;
        const dataTransfer = new DataTransfer();
        dataTransfer.setData('text', text);
        const event = new ClipboardEvent('paste', {{clipboardData: dataTransfer, bubbles: true}});
        arguments[0].dispatchEvent(event)''',
        el)

In [ ]:
def send_to_clipboard(clip_type, image_data):
    win32clipboard.OpenClipboard()
    win32clipboard.EmptyClipboard()
    win32clipboard.SetClipboardData(clip_type, image_data)
    win32clipboard.CloseClipboard()

In [ ]:
def calculate_diff_dates(fecha_str):
    # Convertir la cadena de fecha a un objeto datetime
    fecha_dada = datetime.strptime(fecha_str, "%d/%m/%Y")
    # Obtener la fecha actual
    fecha_actual = datetime.now()
    # Calcular la diferencia en días
    diferencia_dias = (fecha_actual - fecha_dada).days
    
    # Calcular los meses y días
    if diferencia_dias > 0:
        meses = diferencia_dias // 30
        dias = diferencia_dias % 30
    else:
        meses = 0
        dias = diferencia_dias
    
    return meses, dias

In [ ]:
def generate_message_date(fecha_input):
    meses, dias = calculate_diff_dates(fecha_input)
    if meses == 1:
        atraso = f"{meses} mes"
    elif meses > 1:
        atraso = f"{meses} meses"
    else:
        atraso = f"{dias} días"
    return atraso

In [ ]:
## Load image to Bytes
filepath = "images/image_promo.jpg"
image = Image.open(filepath)

output = BytesIO()
image.convert("RGB").save(output, "BMP")
image_data = output.getvalue()[14:]
output.close()

In [ ]:
message_promotion = """
Hola {0}. Se le informa que presenta un atraso de {1} en el pago de cuenta. Su CODIGO de cliente es: *{2}*
"""

In [ ]:
## Bucle
# Lee el archivo CSV en un DataFrame
df = pd.read_csv("data/cobranzas.csv", dtype={'TELEFONO': str, 'ESTADO_ENVIADO': str, 'CODIGO': str})

# Itera sobre cada fila del DataFrame
for index, row in df.iterrows():
    if row['ESTADO_ENVIADO'] == "1" or row['ESTADO_ENVIADO'] == "2":
        continue
    try:
        print(str(row['TELEFONO']))
        time.sleep(0.2)
        browser.find_element(By.XPATH,
                             "//*[@id='app']/div/div[3]/div/div[3]/header/header/div/span/div/span/div[1]/div/span").click()
        time.sleep(0.2)
        browser.find_element(By.XPATH,
                             "//*[@id='app']/div/div[3]/div/div[2]/div[1]/span/div/span/div/div[1]/div[2]/div[2]/div/div/p").send_keys(
            str(row['TELEFONO']))
        time.sleep(1)
        
        if len(browser.find_elements(By.XPATH, "//*[@id='app']/div/div[3]/div/div[2]/div[1]/span/div/span/div/div[2]/div[2]")) == 0:
            browser.find_element(By.XPATH, "//*[@id='app']/div/div[3]/div/div[2]/div[1]/span/div/span/div/header/div/div[1]/div/span").click()
            df.at[index, 'ESTADO_ENVIADO'] = 2
            continue
            
        browser.find_element(By.XPATH, "//*[@id='app']/div/div[3]/div/div[2]/div[1]/span/div/span/div/div[2]/div[2]").click()
        time.sleep(0.2)
        ## Send image
        send_to_clipboard(win32clipboard.CF_DIB, image_data)
        browser.find_element(
            By.XPATH,
            "//*[@id='main']/footer/div[1]/div/span/div/div[2]/div[1]/div/div[1]/p").send_keys(Keys.CONTROL + "v")
        time.sleep(0.3)
        ## Send Text Promotion
        send_text = browser.find_element(By.XPATH,
                                         "//*[@id='app']/div/div[3]/div/div[2]/div[2]/span/div/div/div/div[2]/div/div[1]/div[3]/div/div/div[2]/div[1]/div[1]/p")
        paste_content(browser, send_text, message_promotion.format(row['NOMBRE'], generate_message_date(row['FECHA_VENCIDO']), row['CODIGO']))
        ## Send Message
        browser.find_element(By.XPATH,
                             "//*[@id='app']/div/div[3]/div/div[2]/div[2]/span/div/div/div/div[2]/div/div[2]/div[2]/div/div").click()
        ## Actualizado de csv
        df.at[index, 'ESTADO_ENVIADO'] = 1
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        
# Guardar el DataFrame actualizado en un nuevo archivo CSV
df.to_csv("data/cobranzas.csv", index=False)